In [1]:
import string

In [2]:
class Vectorizer:
    def standardize(self, text):
        text = text.lower()
        return "".join(char for char in text if char not in string.punctuation)
    
    def tokenize(self, text):
        text = self.standardize(text)
        return text.split()
    
    def make_vocab(self, dataset):
        self.vocab = {"": 0, "[UNK]": 1}
        for text in dataset:
            tokens = self.tokenize(text)
            for token in tokens:
                if token not in self.vocab:
                    self.vocab[token] = len(self.vocab)
        self.inverse_vocab = dict(
            (v, k) for k, v in self.vocab.items()
        )
        
    def encode(self, text):
        tokens = self.tokenize(text)
        return [self.vocab.get(token, 1) for token in tokens]
    
    def decode(self, int_sequence):
        return " ".join(
            self.inverse_vocab.get(i, "[UNK]") for i in int_sequence
        )

In [7]:
vectorizer = Vectorizer()
dataset = [
    "Out of the mid-wood's twilight",
    "Into the meadow's dawn",
    "Ivory limbed and brown-eyed",
    "Flashes my Faun!",
]
vectorizer.make_vocab(dataset)

In [8]:
test_sentence = "Out of the mid-wood's dawn there is ivory"
encoded_sentence = vectorizer.encode(test_sentence)
print(encoded_sentence)

[2, 3, 4, 5, 9, 1, 1, 10]


In [9]:
decoded_sentence = vectorizer.decode(encoded_sentence)
print(decoded_sentence)

out of the midwoods dawn [UNK] [UNK] ivory


In [12]:
from tensorflow.keras.layers import TextVectorization
text_vectorizer = TextVectorization(
    output_mode="int"
)

In [13]:
text_vectorizer.adapt(dataset)

In [14]:
text_vectorizer.get_vocabulary()

['',
 '[UNK]',
 'the',
 'twilight',
 'out',
 'of',
 'my',
 'midwoods',
 'meadows',
 'limbed',
 'ivory',
 'into',
 'flashes',
 'faun',
 'dawn',
 'browneyed',
 'and']

In [19]:
vocabulary = text_vectorizer.get_vocabulary()
encoded_sentence = text_vectorizer(test_sentence)
print(encoded_sentence)

tf.Tensor([ 4  5  2  7 14  1  1 10], shape=(8,), dtype=int64)


In [20]:
inverse_vocab = dict(enumerate(vocabulary))
decoded_sentence = " ".join(inverse_vocab[int(i)] for i in encoded_sentence)
print(decoded_sentence)

out of the midwoods dawn [UNK] [UNK] ivory


In [22]:
!cat aclImdb/train/pos/4077_10.txt

I first saw this back in the early 90s on UK TV, i did like it then but i missed the chance to tape it, many years passed but the film always stuck with me and i lost hope of seeing it TV again, the main thing that stuck with me was the end, the hole castle part really touched me, its easy to watch, has a great story, great music, the list goes on and on, its OK me saying how good it is but everyone will take there own best bits away with them once they have seen it, yes the animation is top notch and beautiful to watch, it does show its age in a very few parts but that has now become part of it beauty, i am so glad it has came out on DVD as it is one of my top 10 films of all time. Buy it or rent it just see it, best viewing is at night alone with drink and food in reach so you don't have to stop the film.<br /><br />Enjoy

In [23]:
import os, pathlib, shutil, random

In [25]:
base_dir = pathlib.Path("./aclImdb")
train_dir = base_dir / "train"
val_dir = base_dir / "val"
test_dir = base_dir / "test"
for category in ("neg", "pos"):
    os.makedirs(val_dir / category)
    files= os.listdir(train_dir / category)
    random.Random(42).shuffle(files)
    num_val_samples = int(0.2 * len(files))
    val_files = files[-num_val_samples:]
    for fname in val_files:
        shutil.move(train_dir / category / fname, val_dir / category / fname)

FileExistsError: [Errno 17] File exists: 'aclImdb/val/neg'

In [40]:
from tensorflow import keras

batch_size = 32

train_ds = keras.utils.text_dataset_from_directory(
    train_dir, batch_size=batch_size
)
val_ds = keras.utils.text_dataset_from_directory(
    val_dir, batch_size=batch_size
)
test_ds = keras.utils.text_dataset_from_directory(
    test_dir, batch_size=batch_size
)

Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


In [41]:
inputs, targets = next(iter(train_ds))

In [42]:
print(inputs.shape)
print(inputs.dtype)
print(targets.shape)
print(targets.dtype)
print(inputs[0])
print(targets[0])

(32,)
<dtype: 'string'>
(32,)
<dtype: 'int32'>
tf.Tensor(b'Well, you might not actually SEE any women in love in this movie, but you\'ll certainly hear women TALKING about love, and men talking about love, and women talking about men, and men talking about women, and men talking about men, and everyone talking about death, and talking, and talking, until you yourself will want to scream and do something that requires no talking at all, like paint your bedroom or water your plants.<br /><br />Welcome to the world of D.H. Lawrence, where psycho-babble reigns supreme, and where no one can get down to living a productive life because everyone is too busy talking about how unproductive their lives are. Spending time with the characters in a D.H. Lawrence novel is like being locked in a closet with a group of your most self-absorbed acquaintances who you would run away from if you met them at a party. When I read "Women in Love," I so desperately wanted to strangle every single character in 

In [43]:
text_vectorization = TextVectorization(
    max_tokens=20000,
    output_mode="multi_hot"
)

In [44]:
text_only_train_ds = train_ds.map(lambda x, y: x)
text_vectorization.adapt(text_only_train_ds)

binary_1gram_train_ds = train_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=4)
binary_1gram_val_ds = val_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=4)
binary_1gram_test_ds = test_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=4)

In [45]:
inputs, targets = next(iter(binary_1gram_train_ds))
print(inputs.shape)
print(inputs.dtype)
print(targets.shape)
print(targets.dtype)
print(inputs[0])
print(targets[0])

(32, 20000)
<dtype: 'float32'>
(32,)
<dtype: 'int32'>
tf.Tensor([1. 1. 1. ... 0. 0. 0.], shape=(20000,), dtype=float32)
tf.Tensor(0, shape=(), dtype=int32)


In [46]:
from tensorflow.keras import layers

def get_model(max_tokens=20000, hidden_dim=16):
    inputs = layers.Input(shape=(max_tokens,))
    x = layers.Dense(hidden_dim, activation='relu')(inputs)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(1, activation='sigmoid')(x)
    model = keras.Model(inputs=inputs, outputs=outputs)
    model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    return model

In [47]:
model = get_model()
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 20000)]           0         
                                                                 
 dense_4 (Dense)             (None, 16)                320016    
                                                                 
 dropout_2 (Dropout)         (None, 16)                0         
                                                                 
 dense_5 (Dense)             (None, 1)                 17        
                                                                 
Total params: 320033 (1.22 MB)
Trainable params: 320033 (1.22 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [48]:
callbacks = [keras.callbacks.ModelCheckpoint("binary_1gram.keras", save_best_only=True)]
history = model.fit(binary_1gram_train_ds.cache(), epochs=10, validation_data=binary_1gram_val_ds.cache(), callbacks=callbacks)

Epoch 1/10
625/625 [==============================] - 1s 2ms/step - loss: 0.4135 - accuracy: 0.8248 - val_loss: 0.3123 - val_accuracy: 0.8730
Epoch 2/10
625/625 [==============================] - 1s 2ms/step - loss: 0.2697 - accuracy: 0.9013 - val_loss: 0.3060 - val_accuracy: 0.8828
Epoch 3/10
625/625 [==============================] - 1s 2ms/step - loss: 0.2347 - accuracy: 0.9193 - val_loss: 0.3193 - val_accuracy: 0.8852
Epoch 4/10
625/625 [==============================] - 1s 2ms/step - loss: 0.2163 - accuracy: 0.9280 - val_loss: 0.3537 - val_accuracy: 0.8842
Epoch 5/10
625/625 [==============================] - 1s 2ms/step - loss: 0.2128 - accuracy: 0.9302 - val_loss: 0.3609 - val_accuracy: 0.8878
Epoch 6/10
625/625 [==============================] - 1s 2ms/step - loss: 0.2012 - accuracy: 0.9365 - val_loss: 0.3835 - val_accuracy: 0.8864
Epoch 7/10
625/625 [==============================] - 1s 2ms/step - loss: 0.1941 - accuracy: 0.9384 - val_loss: 0.3960 - val_accuracy: 0.8860
Epoch 

In [52]:
model = keras.models.load_model("binary_1gram.keras")
model.evaluate(binary_1gram_test_ds.cache())

782/782 [==============================] - 1s 1ms/step - loss: 0.2954 - accuracy: 0.8814


[0.2953505218029022, 0.8813999891281128]

In [53]:
text_vectorization = TextVectorization(
    ngrams=2,
    max_tokens=20000,
    output_mode='multi_hot'
)

In [54]:
text_vectorization.adapt(text_only_train_ds)
binary_2gram_train_ds = train_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=4)
binary_2gram_val_ds = val_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=4)
binary_2gram_test_ds = test_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=4)

In [55]:
model = get_model()
callbacks = [keras.callbacks.ModelCheckpoint("binary_2gram.keras", save_best_only=True)]
history = model.fit(binary_2gram_train_ds.cache(), epochs=10, validation_data=binary_2gram_val_ds.cache(), callbacks=callbacks)

Epoch 1/10
625/625 [==============================] - 2s 2ms/step - loss: 0.3752 - accuracy: 0.8451 - val_loss: 0.2721 - val_accuracy: 0.8964
Epoch 2/10
625/625 [==============================] - 1s 2ms/step - loss: 0.2394 - accuracy: 0.9141 - val_loss: 0.2813 - val_accuracy: 0.9020
Epoch 3/10
625/625 [==============================] - 1s 2ms/step - loss: 0.2048 - accuracy: 0.9333 - val_loss: 0.3004 - val_accuracy: 0.9012
Epoch 4/10
625/625 [==============================] - 1s 2ms/step - loss: 0.1824 - accuracy: 0.9431 - val_loss: 0.3356 - val_accuracy: 0.9004
Epoch 5/10
625/625 [==============================] - 1s 2ms/step - loss: 0.1807 - accuracy: 0.9484 - val_loss: 0.3551 - val_accuracy: 0.8984
Epoch 6/10
625/625 [==============================] - 1s 2ms/step - loss: 0.1667 - accuracy: 0.9525 - val_loss: 0.3670 - val_accuracy: 0.8984
Epoch 7/10
625/625 [==============================] - 1s 2ms/step - loss: 0.1594 - accuracy: 0.9542 - val_loss: 0.3715 - val_accuracy: 0.8980
Epoch 

In [56]:
model = keras.models.load_model("binary_2gram.keras")
model.evaluate(binary_2gram_test_ds.cache())

782/782 [==============================] - 1s 1ms/step - loss: 0.2642 - accuracy: 0.8988


[0.26423379778862, 0.8988400101661682]

In [58]:
text_vectorization = TextVectorization(
    ngrams=2,
    max_tokens=20000,
    output_mode='tf_idf'
)

In [59]:
text_vectorization.adapt(text_only_train_ds)

tfidf_2gram_train_ds = train_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=4)
tfidf_2gram_val_ds = val_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=4)
tfidf_2gram_test_ds = test_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=4)

2023-12-28 13:06:07.927430: W tensorflow/compiler/mlir/tools/kernel_gen/transforms/gpu_kernel_to_blob_pass.cc:191] Failed to compile generated PTX with ptxas. Falling back to compilation by driver.


In [60]:
model = get_model()
callbacks = [keras.callbacks.ModelCheckpoint("tfidf_2gram.keras", save_best_only=True)]
model.fit(tfidf_2gram_train_ds.cache(), epochs=10, validation_data=tfidf_2gram_val_ds.cache(), callbacks=callbacks)

Epoch 1/10
625/625 [==============================] - 2s 2ms/step - loss: 0.4996 - accuracy: 0.7562 - val_loss: 0.2876 - val_accuracy: 0.8884
Epoch 2/10
625/625 [==============================] - 1s 2ms/step - loss: 0.3414 - accuracy: 0.8450 - val_loss: 0.2918 - val_accuracy: 0.8904
Epoch 3/10
625/625 [==============================] - 1s 2ms/step - loss: 0.3092 - accuracy: 0.8563 - val_loss: 0.3142 - val_accuracy: 0.8720
Epoch 4/10
625/625 [==============================] - 1s 2ms/step - loss: 0.2889 - accuracy: 0.8691 - val_loss: 0.3283 - val_accuracy: 0.8876
Epoch 5/10
625/625 [==============================] - 1s 2ms/step - loss: 0.2721 - accuracy: 0.8788 - val_loss: 0.3531 - val_accuracy: 0.8892
Epoch 6/10
625/625 [==============================] - 1s 2ms/step - loss: 0.2708 - accuracy: 0.8757 - val_loss: 0.3333 - val_accuracy: 0.8882
Epoch 7/10
625/625 [==============================] - 1s 2ms/step - loss: 0.2607 - accuracy: 0.8871 - val_loss: 0.3409 - val_accuracy: 0.8708
Epoch 

In [61]:
model = keras.models.load_model("tfidf_2gram.keras")
model.evaluate(tfidf_2gram_test_ds.cache())

782/782 [==============================] - 1s 1ms/step - loss: 0.2936 - accuracy: 0.8820


[0.29364439845085144, 0.8820000290870667]

In [62]:
inputs = keras.Input(shape=(1,), dtype='string')
processed_inputs = text_vectorization(inputs)
outputs = model(processed_inputs)
inference_model = keras.Model(inputs=inputs, outputs=outputs)

In [72]:
import tensorflow as tf

raw_text_data = tf.convert_to_tensor(["The movie was amazing, I loved every part of it, would watch again with my friends, "
                                      "I love everything about it! Great!"])
predictions = inference_model(raw_text_data)
print(f"{float(predictions[0]* 100):.2f} % positive")

87.04 % positive


In [73]:
max_length = 600
max_tokens = 20000

text_vectorization = layers.TextVectorization(
    max_tokens=max_tokens,
    output_sequence_length=max_length,
    output_mode="int"
)

text_vectorization.adapt(text_only_train_ds)

int_train_ds = train_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=4)
int_val_ds = val_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=4)
int_test_ds = test_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=4)

In [74]:
inputs = keras.Input(shape=(None,), dtype='int64')
embedded = tf.one_hot(inputs, depth=max_tokens)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation='sigmoid')(x)
model = keras.Model(inputs=inputs, outputs=outputs)
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

2023-12-28 13:20:17.471851: W tensorflow/compiler/mlir/tools/kernel_gen/transforms/gpu_kernel_to_blob_pass.cc:191] Failed to compile generated PTX with ptxas. Falling back to compilation by driver.


Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, None)]            0         
                                                                 
 tf.one_hot (TFOpLambda)     (None, None, 20000)       0         
                                                                 
 bidirectional (Bidirection  (None, 64)                5128448   
 al)                                                             
                                                                 
 dropout_5 (Dropout)         (None, 64)                0         
                                                                 
 dense_10 (Dense)            (None, 1)                 65        
                                                                 
Total params: 5128513 (19.56 MB)
Trainable params: 5128513 (19.56 MB)
Non-trainable params: 0 (0.00 Byte)
___________________

In [75]:
callbacks = [keras.callbacks.ModelCheckpoint("one_hot_bidir_lstm.keras", save_best_only=True)]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=10, callbacks=callbacks)

Epoch 1/10
625/625 [==============================] - 60s 94ms/step - loss: 0.5627 - accuracy: 0.7119 - val_loss: 0.3791 - val_accuracy: 0.8416
Epoch 2/10
625/625 [==============================] - 58s 93ms/step - loss: 0.3576 - accuracy: 0.8630 - val_loss: 0.3353 - val_accuracy: 0.8632
Epoch 3/10
625/625 [==============================] - 58s 93ms/step - loss: 0.2962 - accuracy: 0.8964 - val_loss: 0.2893 - val_accuracy: 0.8876
Epoch 4/10
625/625 [==============================] - 59s 94ms/step - loss: 0.2502 - accuracy: 0.9135 - val_loss: 0.3093 - val_accuracy: 0.8746
Epoch 5/10
625/625 [==============================] - 58s 94ms/step - loss: 0.2189 - accuracy: 0.9262 - val_loss: 0.3427 - val_accuracy: 0.8598
Epoch 6/10
625/625 [==============================] - 59s 94ms/step - loss: 0.1896 - accuracy: 0.9367 - val_loss: 0.3100 - val_accuracy: 0.8780
Epoch 7/10
625/625 [==============================] - 59s 94ms/step - loss: 0.1721 - accuracy: 0.9465 - val_loss: 0.3059 - val_accuracy:

In [76]:
model = keras.models.load_model("one_hot_bidir_lstm.keras")
model.evaluate(int_test_ds)

782/782 [==============================] - 41s 52ms/step - loss: 0.2968 - accuracy: 0.8811


[0.29682227969169617, 0.8810799717903137]

In [77]:
inputs = keras.Input(shape=(None,), dtype='int64')
embedded = layers.Embedding(input_dim=max_tokens, output_dim=256)(inputs)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation='sigmoid')(x)
model = keras.Model(inputs=inputs, outputs=outputs)
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=["accuracy"])

In [78]:
callbacks = [keras.callbacks.ModelCheckpoint("embedding_lstm.keras", save_best_only=True)]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=10, callbacks=callbacks)

Epoch 1/10
625/625 [==============================] - 32s 49ms/step - loss: 0.5189 - accuracy: 0.7473 - val_loss: 0.3960 - val_accuracy: 0.8468
Epoch 2/10
625/625 [==============================] - 24s 39ms/step - loss: 0.3611 - accuracy: 0.8619 - val_loss: 0.3219 - val_accuracy: 0.8740
Epoch 3/10
625/625 [==============================] - 23s 36ms/step - loss: 0.2809 - accuracy: 0.8978 - val_loss: 0.3466 - val_accuracy: 0.8680
Epoch 4/10
625/625 [==============================] - 22s 35ms/step - loss: 0.2326 - accuracy: 0.9165 - val_loss: 0.3376 - val_accuracy: 0.8736
Epoch 5/10
625/625 [==============================] - 21s 34ms/step - loss: 0.1979 - accuracy: 0.9302 - val_loss: 0.3601 - val_accuracy: 0.8654
Epoch 6/10
625/625 [==============================] - 21s 33ms/step - loss: 0.1746 - accuracy: 0.9406 - val_loss: 0.4988 - val_accuracy: 0.8362
Epoch 7/10
625/625 [==============================] - 20s 32ms/step - loss: 0.1443 - accuracy: 0.9543 - val_loss: 0.3866 - val_accuracy:

In [79]:
model = keras.models.load_model("embedding_lstm.keras")
model.evaluate(int_test_ds)

782/782 [==============================] - 10s 11ms/step - loss: 0.3296 - accuracy: 0.8695


[0.3296020030975342, 0.8694800138473511]

In [81]:
inputs = keras.Input(shape=(None, ), dtype='int64')
embedded = layers.Embedding(input_dim=max_tokens, output_dim=256, mask_zero=True)(inputs)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop", loss="binary_crossentropy", metrics=['accuracy'])

In [82]:
callbacks = [keras.callbacks.ModelCheckpoint("embedding_lstm_masked.keras", save_best_only=True)]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=10, callbacks=callbacks)

Epoch 1/10


2023-12-28 13:49:22.883109: W tensorflow/core/common_runtime/type_inference.cc:339] Type inference failed. This indicates an invalid graph that escaped type checking. Error message: INVALID_ARGUMENT: expected compatible input types, but input 1:
type_id: TFT_OPTIONAL
args {
  type_id: TFT_PRODUCT
  args {
    type_id: TFT_TENSOR
    args {
      type_id: TFT_INT32
    }
  }
}
 is neither a subtype nor a supertype of the combined inputs preceding it:
type_id: TFT_OPTIONAL
args {
  type_id: TFT_PRODUCT
  args {
    type_id: TFT_TENSOR
    args {
      type_id: TFT_FLOAT
    }
  }
}

	for Tuple type infernce function 0
	while inferring type of node 'cond_36/output/_23'
2023-12-28 13:49:23.006447: W tensorflow/compiler/mlir/tools/kernel_gen/transforms/gpu_kernel_to_blob_pass.cc:191] Failed to compile generated PTX with ptxas. Falling back to compilation by driver.


625/625 [==============================] - 38s 54ms/step - loss: 0.4536 - accuracy: 0.7785 - val_loss: 0.3578 - val_accuracy: 0.8330
Epoch 2/10
625/625 [==============================] - 26s 41ms/step - loss: 0.2704 - accuracy: 0.8910 - val_loss: 0.2876 - val_accuracy: 0.8788
Epoch 3/10
625/625 [==============================] - 23s 37ms/step - loss: 0.2017 - accuracy: 0.9237 - val_loss: 0.3759 - val_accuracy: 0.8550
Epoch 4/10
625/625 [==============================] - 22s 36ms/step - loss: 0.1562 - accuracy: 0.9417 - val_loss: 0.3083 - val_accuracy: 0.8800
Epoch 5/10
625/625 [==============================] - 22s 34ms/step - loss: 0.1173 - accuracy: 0.9585 - val_loss: 0.4063 - val_accuracy: 0.8670
Epoch 6/10
625/625 [==============================] - 21s 34ms/step - loss: 0.0914 - accuracy: 0.9699 - val_loss: 0.4520 - val_accuracy: 0.8780
Epoch 7/10
625/625 [==============================] - 21s 34ms/step - loss: 0.0694 - accuracy: 0.9768 - val_loss: 0.4466 - val_accuracy: 0.8784
Epo

In [83]:
model = keras.models.load_model("embedding_lstm_masked.keras")
model.evaluate(int_test_ds)

782/782 [==============================] - 14s 15ms/step - loss: 0.2979 - accuracy: 0.8760


[0.2979428172111511, 0.8759999871253967]

In [84]:
import numpy as np
path_to_glove_file = "glove.6B.100d.txt"

embeddings_index = {}

with open(path_to_glove_file) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs
print(f"Found {len(embeddings_index)} word vectors")

Found 400000 word vectors


In [85]:
embedding_dim = 100

vocabulary = text_vectorization.get_vocabulary()
word_index = dict(zip(vocabulary, range(len(vocabulary))))

embeddings_matrix = np.zeros((max_tokens, embedding_dim))
for word, i in word_index.items():
    if i < max_tokens:
        embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embeddings_matrix[i] = embedding_vector

In [86]:
embedding_layer = layers.Embedding(
    max_tokens,
    embedding_dim,
    embeddings_initializer=keras.initializers.Constant(embeddings_matrix),
    trainable=False,
    mask_zero=True
)
inputs = layers.Input(shape=(None,), dtype='int64')
embedded = embedding_layer(inputs)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop", loss="binary_crossentropy", metrics=["accuracy"])

In [87]:
callbacks = [keras.callbacks.ModelCheckpoint("glove_embeddings_sequence.keras", save_best_only=True)]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=10, callbacks=callbacks)

Epoch 1/10
625/625 [==============================] - 34s 49ms/step - loss: 0.5720 - accuracy: 0.6974 - val_loss: 0.4479 - val_accuracy: 0.8010
Epoch 2/10
625/625 [==============================] - 29s 47ms/step - loss: 0.4492 - accuracy: 0.7977 - val_loss: 0.3847 - val_accuracy: 0.8322
Epoch 3/10
625/625 [==============================] - 30s 48ms/step - loss: 0.3965 - accuracy: 0.8261 - val_loss: 0.3540 - val_accuracy: 0.8386
Epoch 4/10
625/625 [==============================] - 29s 47ms/step - loss: 0.3656 - accuracy: 0.8436 - val_loss: 0.3281 - val_accuracy: 0.8608
Epoch 5/10
625/625 [==============================] - 29s 47ms/step - loss: 0.3379 - accuracy: 0.8585 - val_loss: 0.3175 - val_accuracy: 0.8680
Epoch 6/10
625/625 [==============================] - 29s 46ms/step - loss: 0.3182 - accuracy: 0.8654 - val_loss: 0.3092 - val_accuracy: 0.8698
Epoch 7/10
625/625 [==============================] - 21s 34ms/step - loss: 0.3030 - accuracy: 0.8735 - val_loss: 0.3116 - val_accuracy:

In [88]:
model = keras.models.load_model("glove_embeddings_sequence.keras")
model.evaluate(int_test_ds)

782/782 [==============================] - 14s 15ms/step - loss: 0.2921 - accuracy: 0.8772


[0.29214969277381897, 0.877240002155304]